In [1]:
!pip install --upgrade pytube
!pip install --upgrade youtube-dl
!pip install --upgrade yt-dlp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 96.8 MB/s eta 0:00:00


In [2]:
!pip install pytube

In [3]:
!pip install ffprobe

  Preparing metadata (setup.py) ... done
  Created wheel for ffprobe: filename=ffprobe-0.5-py3-none-any.whl size=3408 sha256=8a73f32cf46c9c3b9fa459ff0d35c5ddb7eda0c18fdd8ac26b61a0633b0cc662
  Stored in directory: /root/.cache/pip/wheels/a2/66/e3/5da9a7e12ee519eed653b188eb8dd7ca780f5c882922beb15d
Successfully built ffprobe


In [4]:
import yt_dlp

ydl_opts = {
    'format': 'bestaudio[ext=webm]',  # Select the best audio format with .webm extension
    'outtmpl': 'downloaded_audio.%(ext)s',  # Output filename with the appropriate extension
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'webm',  # Ensure the audio is in webm format
    }],
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=QB1ZqOxfmlw&ab_channel=Intro%C3%A0l%27apprentissageautomatique-ULaval'])


[youtube] Extracting URL: https://www.youtube.com/watch?v=QB1ZqOxfmlw&ab_channel=Intro%C3%A0l%27apprentissageautomatique-ULaval
[youtube] QB1ZqOxfmlw: Downloading webpage
[youtube] QB1ZqOxfmlw: Downloading ios player API JSON
[youtube] QB1ZqOxfmlw: Downloading mweb player API JSON
[youtube] QB1ZqOxfmlw: Downloading player 3ede36f2
[youtube] QB1ZqOxfmlw: Downloading m3u8 information
[info] QB1ZqOxfmlw: Downloading 1 format(s): 251
[download] Destination: downloaded_audio.webm
[download] 100% of    1.76MiB in 00:00:00 at 2.50MiB/s   


KeyError: 'webm'

In [5]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
device

'cuda'

In [15]:
! ffmpeg -i downloaded_audio.webm -vn -ar 16000 -ac 1 -f wav downloaded_audio.wav


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [7]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("automatic-speech-recognition", model="openai/whisper-base",device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Device set to use cuda


In [16]:
dataset = "/content/downloaded_audio.wav"

In [17]:
import numpy as np

target_length_in_m = 2

# convert from minutes to seconds (* 60) to num samples (* sampling rate)
sampling_rate = pipe.feature_extractor.sampling_rate
target_length_in_samples = target_length_in_m * 60 * sampling_rate

# iterate over our streaming dataset, concatenating samples until we hit our target
long_audio = []
for sample in dataset:
    long_audio.extend(sample["audio"]["array"])
    if len(long_audio) > target_length_in_samples:
        break

long_audio = np.asarray(long_audio)

# how did we do?
seconds = len(long_audio) / 16000
minutes, seconds = divmod(seconds, 60)
print(f"Length of audio sample is {minutes} minutes {seconds:.2f} seconds")

TypeError: string indices must be integers

In [18]:
import librosa

In [27]:
dataset = "/content/downloaded_audio.wav"

In [28]:
stream = librosa.stream(
    dataset,
    block_length=30,
    frame_length=16000,
    hop_length=16000,
)

In [29]:
import soundfile as sf
for i,speech in enumerate(stream):
  sf.write(f'{i}.wav',speech,16000)

**Chunk Transcription(4 chunks)**

In [30]:
audio = []
for i in range(4):
  audio.append(f"/content/{i}.wav")

In [31]:
audio

['/content/0.wav', '/content/1.wav', '/content/2.wav', '/content/3.wav']

In [34]:
transcription = pipe(audio)

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


In [36]:
full_transcription = ""
for i in transcription:
  full_transcription += i['text']

In [37]:
full_transcription

' Now we are talking of handling the data sets. So we have in that name space class, we have in the name space tortured cell data, the class data set that is allowing us to manage the data sets. So we should usually define it with a class that implement a method get item. So we can access to an item of the data set easier. And we also have the length method that is written in the size of that data set. And from a data set, we can define a data loader that is an object that is used to give us instances, samples in batches. So we have access to a batch of samples from a given data set. And we need a batch size as well to define the size of these batches. And some arguments are provided for advanced options. data loaders has such a Python iterator. We also have the sub package start vision, which implements the values for functions for computer visions and image processing. So we have a touch vision that data sets that allow us to download several popular data sets such as in this side fo

[model with more than 1024 tokens](https://discuss.huggingface.co/t/which-summarization-model-of-huggingface-supports-more-than-1024-tokens-which-model-is-more-suitable-for-programming-related-articles/25095) the model used contains unlimited input length ?

In [38]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="abertsch/unlimiformer-bart-govreport-alternating")

config.json:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/558M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Device set to use cuda:0


In [39]:
summarized_text = pipe(full_transcription)

In [43]:
summarized_text[0]["generated_text"]

'What GAO Found\n\nData sets allow GAO to manage the data sets. Data'

In [42]:
# Use a pipeline as a high-level helper
from transformers import pipeline

summarizer = pipeline("text2text-generation", model="allenai/PRIMERA")

config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.79G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/197 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/20.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

Device set to use cuda:0


In [44]:
summarized_text = summarizer(full_transcription)

Input ids are automatically padded from 319 to 512 to be a multiple of `config.attention_window`: 512


In [45]:
summarized_text[0]["generated_text"]

'So we have in that name space class, we have the name space tortured cell data, the class'

In [46]:

summarization = pipeline('summarization',model="sshleifer/distilbart-cnn-12-6")

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cuda:0


In [52]:
summarized_text = summarization(full_transcription)

In [53]:
summarized_text[0]['summary_text']

' Data set is class data set that is allowing us to manage the data sets . From a data set, we can define a data loader that is an object that is used to give us instances, samples in batches . So we have access to a batch of samples from a given data set. And we need a batch size as well to define the size of these batches .'

In [57]:
num_iters = int(len(full_transcription)/1000)
summarized_text = []
for i in range(0, num_iters + 1):
  start = 0
  start = i * 1000
  end = (i + 1) * 1000
  print("input text \n" + full_transcription[start:end])
  out = summarization(full_transcription[start:end], min_length = 5, max_length=20)
  out = out[0]
  out = out['summary_text']
  print("Summarized text\n"+out)
  summarized_text.append(out)

print(summarized_text)

input text 
 Now we are talking of handling the data sets. So we have in that name space class, we have in the name space tortured cell data, the class data set that is allowing us to manage the data sets. So we should usually define it with a class that implement a method get item. So we can access to an item of the data set easier. And we also have the length method that is written in the size of that data set. And from a data set, we can define a data loader that is an object that is used to give us instances, samples in batches. So we have access to a batch of samples from a given data set. And we need a batch size as well to define the size of these batches. And some arguments are provided for advanced options. data loaders has such a Python iterator. We also have the sub package start vision, which implements the values for functions for computer visions and image processing. So we have a touch vision that data sets that allow us to download several popular data sets such as in t